In [2]:
dataset = []
with open('Test.txt', 'r') as file:
  dataset = file.readlines()
  print(f'Loaded {len(dataset)} entries')


Loaded 52 entries


In [3]:
import ollama

EMBEDDING_MODEL = 'hf.co/CompendiumLabs/bge-base-en-v1.5-gguf'
LANGUAGE_MODEL = 'hf.co/bartowski/Llama-3.2-1B-Instruct-GGUF'

# Each element in the VECTOR_DB will be a tuple (chunk, embedding)
# The embedding is a list of floats, for example: [0.1, 0.04, -0.34, 0.21, ...]
VECTOR_DB = []

def add_chunk_to_database(chunk):
    # Nhúng chunk và thêm vào VECTOR_DB
    embedding = ollama.embed(model=EMBEDDING_MODEL, input=chunk)['embeddings'][0]
    VECTOR_DB.append((chunk, embedding))

In [ ]:
print(VECTOR_DB[:3]

In [4]:
for i, chunk in enumerate(dataset):
    add_chunk_to_database(chunk.strip())  # Sử dụng strip để loại bỏ ký tự newline
    print(f'Added chunk {i + 1}/{len(dataset)} to the database')

Added chunk 1/52 to the database
Added chunk 2/52 to the database
Added chunk 3/52 to the database
Added chunk 4/52 to the database
Added chunk 5/52 to the database
Added chunk 6/52 to the database
Added chunk 7/52 to the database
Added chunk 8/52 to the database
Added chunk 9/52 to the database
Added chunk 10/52 to the database
Added chunk 11/52 to the database
Added chunk 12/52 to the database
Added chunk 13/52 to the database
Added chunk 14/52 to the database
Added chunk 15/52 to the database
Added chunk 16/52 to the database
Added chunk 17/52 to the database
Added chunk 18/52 to the database
Added chunk 19/52 to the database
Added chunk 20/52 to the database
Added chunk 21/52 to the database
Added chunk 22/52 to the database
Added chunk 23/52 to the database
Added chunk 24/52 to the database
Added chunk 25/52 to the database
Added chunk 26/52 to the database
Added chunk 27/52 to the database
Added chunk 28/52 to the database
Added chunk 29/52 to the database
Added chunk 30/52 to th

In [5]:
def cosine_similarity(a, b):
    dot_product = sum(x * y for x, y in zip(a, b))
    norm_a = sum(x ** 2 for x in a) ** 0.5
    norm_b = sum(x ** 2 for x in b) ** 0.5
    return dot_product / (norm_a * norm_b)

In [26]:
def retrieve(chunk, top_n=3):
    query_embedding = ollama.embed(model=EMBEDDING_MODEL, input=chunk)['embeddings'][0]
    
    # Danh sách tạm thời để lưu trữ cặp (chunk, similarity)
    similarities = []
    
    for chunk, embedding in VECTOR_DB:
        similarity = cosine_similarity(query_embedding, embedding)
        similarities.append((chunk, similarity))
    
    # Sắp xếp theo độ tương đồng giảm dần
    similarities.sort(key=lambda x: x[1], reverse=True)
    
    # Trả về N chunk có độ tương đồng cao nhất
    return similarities[:top_n]

# Nhận truy vấn từ người dùng
input_query = input('Ask me a question: ')
retrieved_knowledge = retrieve(input_query)

print('Retrieved knowledge:')
for chunk, similarity in retrieved_knowledge:
    print(f' - (similarity: {similarity:.2f}) {chunk}')

# Tạo prompt cho chatbot, nhấn mạnh rằng không được tạo ra thông tin mới
instruction_prompt = f'''You are a helpful chatbot.
Use only the following pieces of context to answer the question. Don't make up any new information:
{'\n'.join([f' - {chunk}' for chunk, similarity in retrieved_knowledge])}
'''


# Gọi chatbot để trả lời
stream = ollama.chat(
    model=LANGUAGE_MODEL,
    messages=[
        {'role': 'system', 'content': instruction_prompt},
        {'role': 'user', 'content': input_query},
    ],
    stream=True,
)

# In phản hồi từ chatbot theo thời gian thực
print('Chatbot response:')
for chunk in stream:
    print(chunk['message']['content'], end='', flush=True)


Retrieved knowledge:
 - (similarity: 0.70) Bleach is an action and supernatural anime that aired from 2004 to 2012, following a teenager who becomes a Soul Reaper and battles evil spirits known as Hollows.
 - (similarity: 0.67) Monster is a psychological thriller anime released in 2004, following a doctor who tries to stop a former patient he saved, who has become a serial killer.
 - (similarity: 0.65) Fullmetal Alchemist is an action, adventure, and fantasy anime that aired in 2003, following two brothers who use alchemy to search for the Philosopher's Stone to restore their bodies.
Chatbot response:
Some notable anime series that premiered or aired in 2004 include:

- Hellsing Ultimate (a horror-action series)
- Naruto (an action-comedy series)
- Monster (a psychological thriller series)
- Bleach (an action-supernatural anime)

Note: The year is not specified, as all these shows were released before the start of 2013.